In [4]:
from my_lda_func import *
import nltk
from nltk.corpus import reuters
import lda_class_carol
from plsa import Corpus, Pipeline, Visualize
from plsa.algorithms import PLSA
from plsa.pipeline import DEFAULT_PIPELINE

In [5]:
np.random.seed(1)
ntotal=1000
num_topics=20
documents = reuters.fileids()
documents=np.random.choice(documents,ntotal)
docs=[reuters.raw(d) for d in documents]
labels1=[('earn' in reuters.categories(d)) for d in documents]
labels2=[('grain' in reuters.categories(d)) for d in documents]
tf_df, id2word = tf(docs)


In [6]:
lil = []
for i,row in enumerate(tf_df.values):
    lil_sub = []
    for idx, item in enumerate(row):
        if item:
            lil_sub.append((idx, item))
    lil.append(lil_sub)
res_lda1=my_lda_func(corpus=lil, num_topics=num_topics, id2word=id2word, num_words=5,topics_only=False,passes=1)[1]

performing inference on a chunk of 1000 documents
228/1000 documents converged within 50 iterations
Update topics
topic diff 11.18300607585604


In [4]:
b=lda_class_carol.LDA2(docs)
res_lda2=b.lda(num_topics=num_topics,conv_threshold=1e-2,max_iter=20,npass=1)
res_lda2=np.array(b.gamma)
file = open("res_lda2_20", "wb")
np.save(file, res_lda2)
file.close

/home/jovyan/work/lda_project/lda_class_carol.py:105: UserWarning: Variational inference has not converged. Try more iterations.
  warn('Variational inference has not converged. Try more iterations.')
/home/jovyan/work/lda_project/lda_class_carol.py:157: UserWarning: Newton-Raphson has not converged. Try more iterations.
  warn('Newton-Raphson has not converged. Try more iterations.')


2min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
pipeline = Pipeline(*DEFAULT_PIPELINE)
corpus_plsa=Corpus(docs,pipeline)
plsa=PLSA(corpus_plsa, num_topics, True)
result = plsa.fit()
res_plsa=result.topic_given_doc
file = open("res_plsa_20", "wb")
np.save(file, res_plsa)
file.close

<function BufferedWriter.close>

In [17]:
%%timeit -n 1 -r 1
# bigram
# see https://scikit-learn.org/stable/modules/feature_extraction.html
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(docs)
analyze = vectorizer.build_analyzer()
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)
analyze = bigram_vectorizer.build_analyzer()
res_bigram=bigram_vectorizer.fit_transform(docs).toarray()
file = open("res_bigram_20", "wb")
np.save(file, res_bigram)
file.close

1.31 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
#SVM

In [9]:
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [3]:
file = open("res_plsa_20", "rb")
res_plsa=np.load(file)
file.close

OSError: Failed to interpret file <_io.BufferedReader name='res_plsa_20'> as a pickle

In [18]:
file = open("res_lda2_20", "rb")
res_lda2=np.load(file)
file.close

file = open("res_bigram_20", "rb")
res_bigram=np.load(file)
file.close

<function BufferedReader.close>

In [ ]:
methods='plsa,lda1,lda2,bigram'.split(',')
prop=[0.01,0.05,0.1,0.2]
acclist=[]
nmc=50
for l in range(nmc):
    print(l)
    acc=np.zeros((len(prop),len(methods)))
    for j,p in enumerate(prop):
        for i,x in enumerate([res_plsa,res_lda1,res_lda2,res_bigram]):
            X_train, X_test, y_train, y_test = train_test_split(x, np.array(labels1), test_size=1-p, random_state=l)
            clf = svm.SVC(kernel='linear')
            clf.fit(X_train, y_train)
            clf_predictions = clf.predict(X_test)
            acc[j,i]=clf.score(X_test, y_test)
            print("Accuracy of {}: {}%".format(p,acc[j,i] * 100))
        acclist.append(acc)
file = open("acclist", "wb")
np.save(file, acclist)
file.close

0
Accuracy of 0.01: 62.121212121212125%
Accuracy of 0.01: 63.73737373737374%
Accuracy of 0.01: 58.88888888888889%
Accuracy of 0.01: 63.535353535353536%
Accuracy of 0.05: 77.47368421052632%
Accuracy of 0.05: 66.63157894736842%
Accuracy of 0.05: 74.8421052631579%
Accuracy of 0.05: 85.78947368421052%
Accuracy of 0.1: 82.33333333333334%
Accuracy of 0.1: 74.55555555555556%
Accuracy of 0.1: 75.77777777777777%
Accuracy of 0.1: 91.33333333333333%
Accuracy of 0.2: 83.875%
Accuracy of 0.2: 76.625%
Accuracy of 0.2: 76.5%
Accuracy of 0.2: 93.625%
1
Accuracy of 0.01: 78.98989898989899%
Accuracy of 0.01: 60.80808080808081%
Accuracy of 0.01: 62.82828282828283%
Accuracy of 0.01: 51.61616161616161%
Accuracy of 0.05: 81.89473684210526%
Accuracy of 0.05: 74.94736842105263%
Accuracy of 0.05: 67.6842105263158%
Accuracy of 0.05: 86.21052631578947%
Accuracy of 0.1: 80.33333333333333%
Accuracy of 0.1: 80.11111111111111%
Accuracy of 0.1: 68.22222222222221%
Accuracy of 0.1: 86.11111111111111%
Accuracy of 0.2: 8

In [ ]:
from functools import reduce
acc_avg=np.array(reduce(lambda x,y:x+y, acclist))/len(acclist)
file = open("acc_avg", "wb")
np.save(file, acc_avg)
file.close